<a href="https://colab.research.google.com/github/StefanoGenettiUniTN/appa-chinquinaria/blob/data-visualization/data_visualization.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Data visualization

## Data export bollettino APPA
**Date:** 2025-09-30

**Source:** [link](https://bollettino.appa.tn.it/aria/scarica)

In this section we try to visualize data from bollettino APPA from 2023-01-01 to 2023-03-01. Indeed from the website it is possible to download data within maximum 90 days interval.

In [5]:
%%shell
if [ ! -f "appa-export.csv" ]; then
  gdown -q 1JDDMzu7Jo1polnxJ249Tthim5MaVdTpb -O appa-export.csv
fi